In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import 
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
#Check train
train.isnull().sum()

In [ ]:
#Check test
test.isnull().sum()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score

# Feature Engineering

In [ ]:
full_data = [train, test]

In [ ]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [ ]:
import re

In [ ]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)


In [ ]:
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [ ]:
for dataset in full_data:
    dataset['Cabin'] = dataset['Cabin'].astype(str).str[0]

In [ ]:
train.head()

In [ ]:
y = train['Survived']
X = train.drop(['Survived','Name','Ticket'], axis = 1)

In [ ]:
numerical_features = [c for c, dtype in zip(X.columns, X.dtypes)
                     if dtype.kind in ['i','f'] and c !='PassengerId']
categorical_features = [c for c, dtype in zip(X.columns, X.dtypes)
                     if dtype.kind not in ['i','f']]

In [ ]:
numerical_features

In [ ]:
categorical_features

In [ ]:
#import train_test_split library
from sklearn.model_selection import train_test_split

# create train test split
X_train, X_val, y_train, y_val = train_test_split( X,  y, test_size=0.3, random_state=0, stratify = y)

In [ ]:
preprocessor = make_column_transformer(
    
    (make_pipeline(
    SimpleImputer(strategy = 'median')
        #,KBinsDiscretizer(n_bins=3)
    ), numerical_features),
    
    (make_pipeline(
    SimpleImputer(strategy = 'constant', fill_value = 'missing'),
    OneHotEncoder(categories = 'auto', handle_unknown = 'ignore')), categorical_features),
)

In [ ]:
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_pipeline = make_pipeline(preprocessor,RandomForestClassifier(n_estimators = 200) )

In [ ]:
model_pipeline.fit(X_train, y_train)

In [ ]:
X_prediction = model_pipeline.predict(X_train)

In [ ]:
print(f'Train : {model_pipeline.score(X_train, y_train):.3f}')

In [ ]:
print(f'Test : {model_pipeline.score(X_val, y_val):.3f}')

In [ ]:
submission_prediction = model_pipeline.predict(test.drop(['Name','Ticket'], axis = 1)).astype(int)
#

In [ ]:
AllSub = pd.DataFrame({ 'PassengerId': test['PassengerId'],
                       'Survived' : submission_prediction
    
})

AllSub.to_csv("Solution_Pipeline_RF_IMproved.csv", index = False)